In [1]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision.transforms.v2 as T
import tqdm
import wandb
import cv2
import skdim
import torchvision

sys.path.insert(0, os.path.abspath("python_packages"))
from python_packages import *
from src import *

In [2]:
api = wandb.Api()
model_weights_task1 = api.artifact('hzavadil98/Synthetic data/model-hgjnw6q6:v0', type='model')
model_weights_task2 = api.artifact('hzavadil98/Synthetic data/model-0l4ncfvk:v0', type='model')
model_weights_task1.download()
model_weights_task2.download()

wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  


'/Users/jazav7774/Library/CloudStorage/OneDrive-UiTOffice365/UiT/FYS-8603/artifacts/model-0l4ncfvk:v0'

In [ ]:
model_1 = TwoViewCNN.load_from_checkpoint('artifacts/model-hgjnw6q6:v0/model.ckpt')
model_2 = TwoViewCNN.load_from_checkpoint('artifacts/model-0l4ncfvk:v0/model.ckpt')
train_transform = T.Compose(
   [
       T.RandomHorizontalFlip(0.5),
       T.RandomVerticalFlip(0.5),
       T.GaussianNoise(0.1, 0.1),
   ]
)
train_transform = None
dataloader = Synthetic_2v_Dataloader(
   n_samples=[3000, 1000, 1000], train_transform=train_transform, transform=None, batch_size=32
)

In [5]:
def get_dataset_processed(model, dataloader):
    model.to('mps')
    model.eval()
    all_in_features = []
    all_logits = []
    all_labels = []
    with torch.no_grad():
        for batch in tqdm.tqdm(dataloader, desc="Featurizing dataset"):
            x, y1, y2 = batch
            y = y1 if model.task == 1 else y2
            x = [item.to('mps') for item in x]
            in_features = [resnext(x_i) for resnext, x_i in zip(model.resnexts, x)]
            all_in_features.append(in_features)
            all_logits.append(model.fc(torch.cat(in_features, dim=1)))
            all_labels.append(y)
        all_in_features = [torch.cat([f[i] for f in all_in_features]).cpu().numpy() for i in range(2)]
        all_logits = torch.cat(all_logits, dim=0).cpu().numpy()
        all_labels = torch.cat(all_labels, dim=0).cpu().numpy()
    return all_in_features, all_logits, all_labels

task1_train_features_1, task1_train_logits_1, task1_train_labels_1 = get_dataset_processed(model_1, dataloader.train_dataloader(shuffle=False))

Featurizing dataset: 100%|██████████| 94/94 [01:09<00:00,  1.35it/s]


In [16]:
np.allclose(task1_train_logits_1, task1_train_logits)
np.sum(task1_train_labels_1 == task1_train_labels)
np.allclose(task1_train_features_1[0], task1_train_features[0])

False

In [4]:
loadr = dataloader.train_dataloader()
batch = next(iter(loadr))

In [12]:
model_1.to('mps')
model_1.device
batch = [[batch[0][0].to('mps'), batch[0][1].to('mps')], batch[1].to('mps'), batch[2].to('mps')]
model_1.predict_step(batch)

tensor([[ 1.9511, -2.6446,  0.9064],
        [ 3.0826, -1.6011, -0.6030],
        [-1.3275,  3.0731, -2.1125],
        [ 2.3570, -0.9101, -0.8613],
        [ 1.0499, -0.5063, -0.4077],
        [ 3.4978, -0.8116, -1.4220],
        [-2.5832, -1.6507,  3.0610],
        [-2.0324,  1.4139, -0.1169],
        [-1.5866,  2.2817, -1.4286],
        [-3.0439,  1.0615,  1.7949],
        [-1.5645,  0.4295,  0.7384],
        [ 1.4326,  0.2850, -1.9699],
        [-2.4162, -0.5444,  2.7665],
        [ 2.7365, -1.8712, -0.1407],
        [-0.3608,  0.1120, -0.2543],
        [-0.8106,  1.0472, -1.1978],
        [-3.3814,  1.6017,  2.3988],
        [-1.6339,  3.1603, -2.3954],
        [ 1.4509,  0.3348, -1.9037],
        [-1.5839, -1.4920,  2.2109],
        [ 0.0810,  0.3365, -0.4521],
        [ 1.7463, -3.2812,  1.3192],
        [-1.2392, -0.4792,  0.9002],
        [-3.5499,  3.3755,  0.3926],
        [-2.6563,  1.5992,  0.4666],
        [-1.5535,  1.3608, -0.0809],
        [-0.9517, -0.6679,  1.2819],
 